In [1]:
import numpy as np
import cv2
import pandas as pd
import os
import pickle

# Face Detection Model
- Load the model

In [2]:
face_detection_model = './models/res10_300x300_ssd_iter_140000.caffemodel'
face_detection_proto = './models/deploy.prototxt.txt'
face_descriptor = './models/openface.nn4.small2.v1.t7'
# Load models using cv2 dnn
detector_model = cv2.dnn.readNetFromCaffe(face_detection_proto,face_detection_model)
description_model = cv2.dnn.readNetFromTorch(face_descriptor)

In [3]:
!ls images

Angelina Jolie     Cristiano Ronaldo  Joe Biden          Robert Downey Jr
Barack Obama       Donald Trump       Leonardo DiCaprio  Scarlett Johansson


In [ ]:
# consider sample image
img = cv2.imread('./images/Joe Biden/joe103.jpg')
cv2.imshow('Joe Biden',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

2023-11-29 13:44:07.748 Python[42315:1431702] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


In [ ]:
# step-1: face detection 
# image =  img.copy()
# h,w = image.shape[:2]
# img_blob = cv2.dnn.blobFromImage(image,1,(300,300),(104,177,123),swapRB=False,crop =False)
# set the input
# detector_model.setInput(img_blob)
# detections = detector_model.forward()

# if len(detections) >0:
#     i = np.argmax(detections[0,0,:,2]) # consider the face with max confidence score 
#     confidence = detections[0,0,i,2]
#     if confidence >0.5:
#         box = detections[0,0,i,3:7]*np.array([w,h,w,h])
#         (startx,starty,endx,endy) = box.astype('int')
        # just for reference: draw bounding 
#         img_draw = image.copy()
#         cv2.rectangle(img_draw,(startx,starty),(endx,endy),(0,255,0))

#         # step -2: feauture extraction or embedding
#         roi = image[starty:endy,startx:endx].copy()
#         # get the face descriptors
#         faceblob = cv2.dnn.blobFromImage(roi,1/255,(96,96),(0,0,0),swapRB = True,crop = True)
#         description_model.setInput(faceblob)
#         vectors = description_model.forward()
#         print(vectors)
        
# cv2.imshow('sample',img)
# cv2.imshow('draw_image',img_draw)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [ ]:
def helper(image_path):
    img = cv2.imread(image_path)
    # step-1: face detection 
    image =  img.copy()
    h,w = image.shape[:2]
    img_blob = cv2.dnn.blobFromImage(image,1,(300,300),(104,177,123),swapRB=False,crop =False)
    # set the input
    detector_model.setInput(img_blob)
    detections = detector_model.forward()

    if len(detections) >0:
        i = np.argmax(detections[0,0,:,2]) # consider the face with max confidence score 
        confidence = detections[0,0,i,2]
        if confidence >0.5:
            box = detections[0,0,i,3:7]*np.array([w,h,w,h])
            (startx,starty,endx,endy) = box.astype('int')
            # just for reference: draw bounding 
    #         img_draw = image.copy()
    #         cv2.rectangle(img_draw,(startx,starty),(endx,endy),(0,255,0))

            # step -2: feauture extraction or embedding
            roi = image[starty:endy,startx:endx].copy()
            # get the face descriptors
            faceblob = cv2.dnn.blobFromImage(roi,1/255,(96,96),(0,0,0),swapRB = True,crop = True)
            description_model.setInput(faceblob)
            vectors = description_model.forward()
            
            return vectors # if the face is detected or None
        
    return None


In [ ]:
helper('./images/Joe Biden/joe103.jpg')

In [ ]:
data = dict(data=[], label=[])

# apply helper function to all images and get face descriptors

In [ ]:
folders = os.listdir('images')
for folder in folders:
    filenames = os.listdir('images/{}'.format(folder))
    for filename in filenames:
        try:
            vector = helper('./images/{}/{}'.format(folder,filename))
            if vector is not None:
                data['data'].append(vector)
                data['label'].append(folder)
                print('Feature extracted successfully')
                
        except:
            pass
                        

In [ ]:
data.keys()

In [ ]:
pd.Series(data['label']).value_counts()

In [ ]:
# save the data 
pickle.dump(data,open('data_face_features.pickle',mode='wb'))